# Fortnite to Pubg
Inspired from this [blog](https://towardsdatascience.com/turning-fortnite-into-pubg-with-deep-learning-cyclegan-2f9d339dcdb0)

In [ ]:
import os
import cv2
import numpy as np
from datetime import datetime

from keras_contrib.layers.normalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, Add
from keras.layers import BatchNormalization, Activation, MaxPooling2D, Conv2DTranspose
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, model_from_json
from keras.optimizers import Adam
from keras.initializers import RandomNormal
import sys

In [ ]:
CHECKPOINT = "checkpoint/"
SAVED_IMAGES_PATH = "saved_images/"
LOG_PATH = "checkpoint/log.txt"

EPOCHS = 100
BATCH_SIZE = 1

# Saves Model in every N minutes
TIME_INTERVALS = 20 

# Show Summary of Models
SHOW_SUMMARY = True

# Learning rate of Generator and Discriminator
DISCRIMINATOR_LR_RATE = 0.0002
GENERATOR_LR_RATE = 0.0002

In [ ]:
# Input Image Dimensions
IMAGE_ROWS = 256
IMAGE_COLS = 256
IMAGE_CHANNELS = 3
IMAGE_SHAPE = (IMAGE_ROWS, IMAGE_COLS, IMAGE_CHANNELS)

## Load Data

In [ ]:
from dataloader import Data
data = Data()

## Build Models

### Build Discriminator
#### We are going to build [PatchGAN](https://image.slidesharecdn.com/07-image-to-imagetranslationwithconditionaladversarialnetworks-161125155412/95/imagetoimage-translation-with-conditional-adversarial-nets-upc-reading-group-24-638.jpg?cb=1480089468) Discriminator

In [ ]:
d_layers = 3 # discriminator size
d_filter_size = 64 # filter size of first layer
d_dropout = 0.4
d_loss = 'mse' # mean squared error
d_optimizer = Adam(DISCRIMINATOR_LR_RATE, 0.5)

In [ ]:
# Calculate output shape of D (PatchGAN)
patch = int(IMAGE_ROWS / 2**(d_layers))
patch = (patch, patch, 1)
print("Patch shape: ", patch)

In [ ]:
def build_d_layer(layer_input, filters, f_size=4, strides =2, normalization=True, dropout_rate=d_dropout):
    d = Conv2D(filters, kernel_size=f_size, strides=strides, padding='same')(layer_input)
    d = LeakyReLU(alpha=0.2)(d)
    if dropout_rate:
        d = Dropout(dropout_rate)(d)
    if normalization:
        d = InstanceNormalization()(d)
    return d
    
    
def build_discriminator():
    image = Input(shape=IMAGE_SHAPE)
    layers  = []
    
    # Making  N (d_layers) Layers 
    for i in range(d_layers):
        filter_size = d_filter_size * (2 ** i)
        if not i:
            layer = build_d_layer(image, filter_size, normalization=False)
        else:
            layer = build_d_layer(layers[-1], filter_size)
            
        layers.append(layer)
    layers.append(layer)    
    
    confidence = Conv2D(1, kernel_size=4, strides=1,activation='sigmoid', padding='same')(layers[-1])
    
    # Model(input, output)
    return Model(image, confidence)

In [ ]:
# Discriminator Initialization
DCRM_A = build_discriminator()
DCRM_B = build_discriminator()

# Compile the Discriminator Model
DCRM_A.compile(loss=d_loss, optimizer=d_optimizer, metrics=['accuracy'])
DCRM_B.compile(loss=d_loss, optimizer=d_optimizer, metrics=['accuracy'])

# Show Summary
if SHOW_SUMMARY:
    print('Summary DCRM_A:')
    DCRM_A.summary()
    print('Summary DCRM_B:')
    DCRM_B.summary()


### Build Generator
#### We are going to build a [U-Net ](https://cdn-images-1.medium.com/max/953/1*Z98NhzbVISHa4CoemZS4Kw.png) Generator

In [ ]:
g_filter_size = 64 # filter size of first layer
g_dropout = 0.2
dropout = 0.25
g_optimizer = Adam(GENERATOR_LR_RATE, 0.5)

In [ ]:
def build_conv2d(layer_input, filters, strides=2, f_size=4):
    g = Conv2D(filters, kernel_size=f_size, strides=strides, padding='same')(layer_input)
    g = LeakyReLU(alpha=0.2)(g)
    g = InstanceNormalization()(g)
    return g


def build_deconv2d(layer_input, skip_input, filters, strides=2, f_size=4, dropout_rate=g_dropout):
    g = Conv2DTranspose(filters, kernel_size=f_size, strides=strides, padding='same')(layer_input)
    g = LeakyReLU(alpha=0.2)(g)
    if dropout_rate:
        g = Dropout(dropout_rate)(g)
    g = InstanceNormalization()(g)
    g = Concatenate()([g, skip_input])
    return g


def build_generator():
    image = Input(shape=IMAGE_SHAPE)
    layers = []
    
    #  DownSampling the layers
    conv1 = build_conv2d(image, g_filter_size)
    conv2 = build_conv2d(conv1, g_filter_size*2)
    conv3 = build_conv2d(conv2, g_filter_size*4)
    conv4 = build_conv2d(conv3, g_filter_size*8)
    conv5 = build_conv2d(conv4, g_filter_size*16)
    
    # UpSampling the Layers
    deconv1 = build_deconv2d(conv5, conv4, g_filter_size*8)
    deconv2 = build_deconv2d(deconv1, conv3, g_filter_size*4)
    deconv3 = build_deconv2d(deconv2, conv2, g_filter_size*2)
    deconv4 = build_deconv2d(deconv3, conv1, g_filter_size)
    deconv5 = UpSampling2D(size=2)(deconv4)
    
    generated_image = Conv2D(IMAGE_CHANNELS, kernel_size=4, strides=1, padding='same', activation='tanh')(deconv5)
    
    return Model(image, generated_image)
        

In [ ]:
# Generator Initialization
GEN_AB = build_generator()
GEN_BA = build_generator()

# Generator input PlaceHolder
IMG_A = Input(shape=IMAGE_SHAPE)
IMG_B = Input(shape=IMAGE_SHAPE)

# Generator Summary
if SHOW_SUMMARY:
    print('GEN_AB')
    GEN_AB.summary()
    print('GEN_BA')
    GEN_BA.summary()
    

### Adverserial Net (Combined Network) CYCLE 

In [ ]:
# Prepare fake images from Generator
FAKE_IMG_A = GEN_BA(IMG_B)
FAKE_IMG_B = GEN_AB(IMG_A)

# Reconstruct fake images back to original
RECONSTRUCT_A = GEN_BA(FAKE_IMG_B)
RECONSTRUCT_B = GEN_AB(FAKE_IMG_A)

# Original Idendity of the Image
ID_A = GEN_BA(IMG_A)
ID_B = GEN_AB(IMG_B)

# Discriminator Model shouldn't be affected during Adverserial(Combined Model) Optimization
# So the discriminator model is frozen 
DCRM_A.trainable = False
DCRM_B.trainable = False

# Discriminator Confidence of Fake images
CONF_FAKE_IMG_A = DCRM_A(FAKE_IMG_A)
CONF_FAKE_IMG_B = DCRM_B(FAKE_IMG_B)


#### Initialize Adverserial Net - Combined Network - Cycle

In [ ]:
'''
The Combined model will calculate all this loss:
1) Discriminator loss of generated image
2) Reconstruction loss from generated image back to the original image
3) Identity loss from original image to generated image

This loss will be used to optimize the Generator Model
'''
COMBINED = Model([IMG_A, IMG_B],
                [CONF_FAKE_IMG_A, CONF_FAKE_IMG_B,
                 RECONSTRUCT_A, RECONSTRUCT_B,
                 ID_A, ID_B])

COMBINED.compile(loss= ['mse', 'mse',
                        'mae', 'mae',
                        'mae', 'mae'], 
                 loss_weights = [1, 1,
                                 10.0, 10.0,
                                 1.0, 1.0],
                 optimizer = g_optimizer)

## Utilities
* Saving Model
* Loading Model
* Saving Log
* Save Image

In [ ]:
# Saving Discriminator and Generator Models at Checkpoint Path
def save_model():
    global DCRM_A, DCRM_B, GEN_AB, GEN_BA
    models = [DCRM_A, DCRM_B, GEN_AB, GEN_BA]
    model_names = ['DCRM_A', 'DCRM_B', 'GEN_AB', 'GEN_BA']

    for model, model_name in zip(models, model_names):
        model_path =  CHECKPOINT + "%s.json" % model_name
        weights_path = CHECKPOINT + "/%s.hdf5" % model_name
        options = {"file_arch": model_path, 
                    "file_weight": weights_path}
        json_string = model.to_json()
        open(options['file_arch'], 'w').write(json_string)
        model.save_weights(options['file_weight'])
    print("Saved Model")

# Loading Discriminator and Generator Models from CHECKPOINT Path
def load_model():
    # Checking if all the model exists
    model_names = ['DCRM_A', 'DCRM_B', 'GEN_AB', 'GEN_BA']
    files = os.listdir(CHECKPOINT)
    for model_name in model_names:
        if model_name+".json" not in files or\
           model_name+".hdf5" not in files:
            print("Models not Found")
            return
    
    global DCRM_A, DCRM_B, GEN_AB, GEN_BA, d_optimizer, d_loss
    optimizer = Adam(0.0002, 0.5)
    
    # load DCRM_A Model
    model_path = CHECKPOINT + "%s.json" % 'DCRM_A'
    weight_path = CHECKPOINT + "%s.hdf5" % 'DCRM_A'
    with open(model_path, 'r') as f:
        DCRM_A = model_from_json(f.read())
    DCRM_A.load_weights(weight_path)
    DCRM_A.compile(loss=d_loss, optimizer=d_optimizer, metrics=['accuracy'])
    
    # load DCRM_B Model
    model_path = CHECKPOINT + "%s.json" % 'DCRM_B'
    weight_path = CHECKPOINT + "%s.hdf5" % 'DCRM_B'
    with open(model_path, 'r') as f:
        DCRM_B = model_from_json(f.read())
    DCRM_B.load_weights(weight_path)
    DCRM_B.compile(loss=d_loss, optimizer=d_optimizer, metrics=['accuracy'])
    
    # # load GEN_AB Model
    model_path = CHECKPOINT + "%s.json" % 'GEN_AB'
    weight_path = CHECKPOINT + "%s.hdf5" % 'GEN_AB' 
    with open(model_path, 'r') as f:
        GEN_AB = model_from_json(f.read())
    GEN_AB.load_weights(weight_path)

    #load GEN_BA Model
    model_path = CHECKPOINT + "%s.json" % 'GEN_BA'
    weight_path = CHECKPOINT + "%s.hdf5" % 'GEN_BA'
    with open(model_path, 'r') as f:
         GEN_BA = model_from_json(f.read())
    GEN_BA.load_weights(weight_path)
        
    print("Loaded Model")

# Saving log in LOG_PATH
def save_log(log):
    with open(LOG_PATH, 'a') as f:
        f.write("%s\n" %log)
    
# Save images to SAVE_IMAGE directory
#          #################################
# img =    #IMG_A, FAKE_A, RECONSTRUCTED_A#
#          #IMG_B, FAKE_B, RECONSTRUCTED_B#
           ################################
def save_image(epoch, steps):
    TEST_DATA_A, TEST_DATA_B = data.get_data(1)
    for i in range(TEST_DATA_A.shape[0]):
        original_A = TEST_DATA_A[i]
        original_B = TEST_DATA_B[i]
        real_A = original_A / 127.5 - 1
        real_B = original_B / 127.5 - 1
        real_A = np.reshape(real_A, [1, real_A.shape[0], real_A.shape[1], real_A.shape[2]])
        real_B = np.reshape(real_B, [1, real_B.shape[0], real_B.shape[1], real_B.shape[2]])
        
        fake_A = GEN_BA.predict(real_B)
        fake_A = (fake_A + 1) * 127.5
        fake_A = fake_A.astype(np.uint8)
        fake_B = GEN_AB.predict(real_A)
        fake_B = (fake_B + 1) * 127.5
        fake_B = fake_B.astype(np.uint8)
        
        recon_A = GEN_BA.predict(fake_B)
        recon_A = (recon_A + 1) * 127.5
        recon_A = recon_A.astype(np.uint8)
        recon_B = GEN_AB.predict(fake_A)
        recon_B = (recon_B + 1) * 127.5
        recon_B = recon_B.astype(np.uint8)
        
        image_1 = np.concatenate((original_A, fake_B[0], recon_A[0]), axis=1)
        image_2 = np.concatenate((original_B, fake_A[0], recon_B[0]), axis=1)
        image = np.concatenate((image_1, image_2), axis=0)
        image_path = SAVED_IMAGES_PATH + "_".join([str(epoch), str(steps)]) + ".jpg"
        cv2.imwrite(image_path, image)

## Train Model

In [ ]:
def train():
    start_time = datetime.now()
    saved_time = start_time
    
    for epoch in range(EPOCHS):
        steps = 1
        while True:
            
            imgs_A, imgs_B = data.get_data(BATCH_SIZE)
            
            # If value is None it means it is out of data,
            # it auto resets the data loader and breaks out of 'while' loop and starts as next batch
            if imgs_A is None or imgs_B is None:
                break
            
            # assign local batch_size 
            batch_size = imgs_A.shape[0]
            
            # Rescale the image value from 255 => -1 to 1
            imgs_A = imgs_A / 127.5 - 1
            imgs_B = imgs_B / 127.5 - 1
            
            # Discriminator Ground Truth
            real = np.ones((batch_size,) + patch)
            fake = np.zeros((batch_size,) + patch)
            
            # Train Discriminator
            fake_A = GEN_BA.predict(imgs_B)
            fake_B = GEN_AB.predict(imgs_A)
            
            dA_loss_real = DCRM_A.train_on_batch(imgs_A, real)
            dA_loss_fake = DCRM_A.train_on_batch(fake_A, fake)
            dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)
            
            dB_loss_real = DCRM_B.train_on_batch(imgs_B, real)
            dB_loss_fake = DCRM_B.train_on_batch(fake_B, fake)
            dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)
            
            d_loss = 0.5 * np.add(dA_loss, dB_loss)
            
            
            # Train Generator
            # Training the Generator twice as to catch up with the discriminator
            for i in range(2):
                g_loss = COMBINED.train_on_batch([imgs_A, imgs_B],
                                                 [real, real,
                                                  imgs_A, imgs_B,
                                                  imgs_A, imgs_B])

            
            # Save Model
            current_time = datetime.now()
            difference_time = current_time - saved_time
            if difference_time.seconds >= (TIME_INTERVALS * 60):
                save_model()
                save_image(epoch, steps)
                saved_time = current_time
            display(imgs_A, fake_A, imgs_B, fake_B)
            
            # Print and Save Log
            log = "Ep: %d, steps: %d, D loss: %f, acc: %3d%%, G loss: %f" %(epoch,
                                                                                steps, d_loss[0], 100*d_loss[1],
                                                                                g_loss[0])
            print(log)
            save_log(log)
            steps += 1

In [ ]:
## Loads model if exist in Checkpoint Path
load_model()

In [ ]:
train()